In [13]:
import os, shutil
import numpy as np

from matplotlib import image, pyplot
from skimage.transform import resize


from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout

from keras import callbacks
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, f1_score, recall_score,classification_report,roc_curve, auc 
from sklearn.utils import class_weight

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from PIL import Image,ImageFile
from tensorflow.keras import models
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import optimizers

In [25]:
resultPath = 'C:/Users/HP/Downloads/Results/CNN'

In [2]:
DATA_PATH = "C:/Users/HP/Downloads/Image_Hunter_dataset/"
spamData=[]
label=[]
hashList=[]
count=-1
for folder in os.listdir(DATA_PATH):
    print(">>>Reading ",folder)
    count+=1
    
    for file in os.listdir(DATA_PATH+folder):
        if(str(file).endswith('.jpg') or str(file).endswith('.JPG') or str(file).endswith('.jpeg') or str(file).endswith('.JPEG')):
            img = image.imread(DATA_PATH+folder+'/'+file)
            hsh = hash(tuple(np.array(img).flatten()))
            if(hsh not in hashList):
                spamData.append(resize(img, (156, 156, 3)))
                hashList.append(hsh)
                label.append(count)
spamData=np.array(spamData)
label=np.array(label)

>>>Reading  NaturalImages
>>>Reading  SpamImages


In [3]:
print("Number of SPAM",len(label[label==0]))
print("Number of HAM",len(label[label==1]))

Number of SPAM 810
Number of HAM 895


In [4]:
print("Spam data shape : ",spamData.shape," Label shape : ",label.shape)

Spam data shape :  (1705, 156, 156, 3)  Label shape :  (1705,)


In [5]:
test_valid_Percentage = 0.3
x_train,x_test,y_train,y_test = train_test_split(spamData,label,test_size = test_valid_Percentage,random_state=42, stratify=label,shuffle=True)

In [6]:
print("x_train shape : ",x_train.shape," y_train shape : ",y_train.shape)
print("x_test shape : ",x_test.shape," y_test shape : ",y_test.shape)

x_train shape :  (1193, 156, 156, 3)  y_train shape :  (1193,)
x_test shape :  (512, 156, 156, 3)  y_test shape :  (512,)


In [7]:
print("Number of train SPAM",len(y_train[y_train==0]))
print("Number of train HAM",len(y_train[y_train==1]))

print("Number of test SPAM",len(y_test[y_test==0]))
print("Number of test HAM",len(y_test[y_test==1]))

Number of train SPAM 567
Number of train HAM 626
Number of test SPAM 243
Number of test HAM 269


In [8]:
y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

print("y_train_oh shape : ",y_train_oh.shape," y_test_oh shape : ",y_test_oh.shape)

y_train_oh shape :  (1193, 2)  y_test_oh shape :  (512, 2)


In [32]:
#Initializing Exception Network as Base Model for Transfer Learning

from tensorflow.keras.applications.xception import Xception

base_model_xcp = Xception(include_top=False, weights='imagenet',
            input_shape=(156, 156,3), classes= 2)
for layer in base_model_xcp.layers[:115]:
    layer.trainable = False
for layer in base_model_xcp.layers[116:]:
    layer.trainable = True

for layer in base_model_xcp.layers:
    print(layer.name , layer.trainable)
len(base_model_xcp.layers)

input_5 False
block1_conv1 False
block1_conv1_bn False
block1_conv1_act False
block1_conv2 False
block1_conv2_bn False
block1_conv2_act False
block2_sepconv1 False
block2_sepconv1_bn False
block2_sepconv2_act False
block2_sepconv2 False
block2_sepconv2_bn False
conv2d_16 False
block2_pool False
batch_normalization_16 False
add_48 False
block3_sepconv1_act False
block3_sepconv1 False
block3_sepconv1_bn False
block3_sepconv2_act False
block3_sepconv2 False
block3_sepconv2_bn False
conv2d_17 False
block3_pool False
batch_normalization_17 False
add_49 False
block4_sepconv1_act False
block4_sepconv1 False
block4_sepconv1_bn False
block4_sepconv2_act False
block4_sepconv2 False
block4_sepconv2_bn False
conv2d_18 False
block4_pool False
batch_normalization_18 False
add_50 False
block5_sepconv1_act False
block5_sepconv1 False
block5_sepconv1_bn False
block5_sepconv2_act False
block5_sepconv2 False
block5_sepconv2_bn False
block5_sepconv3_act False
block5_sepconv3 False
block5_sepconv3_bn False

132

In [33]:
NO_OF_EPOCHS=10
BATCH_SIZE=64

model=Sequential()
model.add(base_model_xcp)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 flatten_5 (Flatten)         (None, 51200)             0         
                                                                 
 dense_14 (Dense)            (None, 128)               6553728   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 27,415,337
Trainable params: 13,342,241
Non-trainable params: 14,073,096
_________________________________________________________________


In [34]:
checkpointer = callbacks.ModelCheckpoint(filepath=resultPath+"/checkpoint-{epoch:04d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger(resultPath+'/result_logger.csv',separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.2, patience=2, min_lr=0.001)
model.fit(x_train,y_train,epochs= 10,verbose=1,batch_size= 64,validation_data=(x_test,y_test),callbacks=[checkpointer,csv_logger,reduce_lr])

Epoch 1/10
19/19 [==============================] - 116s 6s/step - loss: 0.2726 - accuracy: 0.9346 - val_loss: 0.8747 - val_accuracy: 0.9609 - lr: 0.0010
Epoch 2/10
19/19 [==============================] - 104s 6s/step - loss: 0.0797 - accuracy: 0.9933 - val_loss: 2.5075 - val_accuracy: 0.9004 - lr: 0.0010
Epoch 3/10
19/19 [==============================] - 105s 6s/step - loss: 0.0302 - accuracy: 0.9933 - val_loss: 0.8246 - val_accuracy: 0.9844 - lr: 0.0010
Epoch 4/10
19/19 [==============================] - 105s 6s/step - loss: 0.0103 - accuracy: 0.9983 - val_loss: 1.1295 - val_accuracy: 0.9336 - lr: 0.0010
Epoch 5/10
19/19 [==============================] - 105s 6s/step - loss: 0.0012 - accuracy: 0.9992 - val_loss: 0.8762 - val_accuracy: 0.9277 - lr: 0.0010
Epoch 6/10
19/19 [==============================] - 105s 6s/step - loss: 0.0054 - accuracy: 0.9983 - val_loss: 0.4074 - val_accuracy: 0.9844 - lr: 0.0010
Epoch 7/10
19/19 [==============================] - 104s 6s/step - loss: 0.0

In [35]:
prediction_prob1 = model.predict(x_test,verbose=1)
y_pred=np.round(prediction_prob1)

16/16 [==============================] - 32s 1s/step


In [36]:
print(accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred,average='binary'))
print(recall_score(y_test,y_pred,average='binary'))
print(f1_score(y_test,y_pred,average='binary'))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.98046875
0.9743589743589743
0.9888475836431226
0.9815498154981549
[[236   7]
 [  3 266]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       243
           1       0.97      0.99      0.98       269

    accuracy                           0.98       512
   macro avg       0.98      0.98      0.98       512
weighted avg       0.98      0.98      0.98       512

